# Links
Video: https://photos.app.goo.gl/H81C6VeLhSQbjzij6


Repo: https://github.com/joedajo/wes267a/tree/main/hw4


# Setup
Here I import libraries and set up the overlay. Then I initialize the Pmod library

In [1]:
import socket
import multiprocessing

from time import sleep

In [2]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

from pynq.lib.pmod.pmod_io import Pmod_IO

io = Pmod_IO(base.PMODA, 3, "out")

# Tone
This is a function that plays a given tone using the overlay

In [3]:
def tone(io, freq: int):
    for _ in range(1000):
        io.write(1)
        sleep(1 / (2 * freq))
        io.write(0)
        sleep(1 / (2 * freq))

In [70]:
tone(io, 100)

In [5]:
base.buttons.read()

0

# Server
The server binds to the PYNQ's address and port 12345. It accepts a connection and accepts data for as long as the connection is open. Any time it receives data, it will play a tone. Once the connection is closed the server will wait for a new one.

In [93]:
def server():
    HOST="192.168.2.99" # pynq
    PORT=12345
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((HOST, PORT))
        s.listen(1)
        while True:
            # get connection
            conn, addr = s.accept()
            print(f"Connection from {addr}")
            with conn:
                while True:
                    # block on recv
                    if not conn.recv(1024):
                        break
                    tone(io, 1000)

# Client
The client does the button polling. When the first button is pressed, a connection is established back to my host machine. My host machine is running it's own server that accepts a connection and forwards the data received back to the PYNQ board. When button two is pressed, the PYNQ sends data to the host. When button three is pressed, the connection to the host machine is closed.

In [94]:
def client():
    HOST="192.168.2.96" # host
    PORT=12345

    s = None
    while True:
        pressed = base.buttons.read()
        if pressed == 1 and s is None:
            print(f"Connecting to {HOST}:{PORT} ... s={s}")
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            try:
                s.connect((HOST, PORT))
            except:
                print(f"Connection failed... resetting socket")
                s = None
                continue
            print(f"Connected...")
        if pressed == 2 and s is not None:
            print(f"Button 2 pressed... sending")
            s.sendall(b'beep')
        if pressed == 4 and s is not None:
            print(f"Button 3 pressed... closing")
            s.close()
            s = None
        sleep(0.1)

# Multiprocessing
Here we dispatch two processes, one for client and one for the server

In [98]:
server_proc = multiprocessing.Process(target=server)
client_proc = multiprocessing.Process(target=client)

In [99]:
client_proc.start()
server_proc.start()

Connecting to 192.168.2.96:12345 ... s=None
Connected...Connection from ('192.168.2.96', 56444)

Button 2 pressed... sending
Button 2 pressed... sending
Button 2 pressed... sending
Button 2 pressed... sending
Button 2 pressed... sending
Button 3 pressed... closing
Connecting to 192.168.2.96:12345 ... s=None
Connected...Connection from ('192.168.2.96', 60746)

Button 2 pressed... sending
Button 3 pressed... closing
Connecting to 192.168.2.96:12345 ... s=None
Connected...Connection from ('192.168.2.96', 58986)

Button 2 pressed... sending
Button 2 pressed... sending
Button 3 pressed... closing


In [97]:
client_proc.join()
server_proc.join()

# Host Machine
This is the code on the host machine. It accepts connections from the PYNQ board's client and forwards it back to the server runnning on the board

```python
import socket


def server():
    HOST="192.168.2.96" # host
    PYNQ="192.168.2.99"
    PORT=12345

    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((HOST, PORT))
        s.listen(1)
        while True:
            conn, addr = s.accept()
            print(f"Connection from {addr}")
            pynqsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            pynqsock.connect((PYNQ, PORT))
            with conn:
                while True:
                    if not conn.recv(1024):
                        break
                    pynqsock.send(b"beep")
                pynqsock.close()

                
server()
```